<div class="Markdown base-markdown base-markdown_with-gallery markdown markdown_size_normal markdown_type_theory full-markdown"><h1>Проект для «Викишоп»</h1><div class="paragraph">Поздравляем! Вы прошли курс в тренажёре. Самое время проверить знания и решить новую задачу машинного обучения. Выполнять работу будете самостоятельно.  </div><div class="paragraph">Когда закончите, отправьте её на проверку ревьюеру: он пришлёт комментарии в течение суток. После этого нужно доработать проект и пройти повторную проверку. </div><div class="paragraph">Скорее всего, вы доработаете кейс по комментариям ещё несколько раз. Это нормально. </div><div class="paragraph">Проект завершён, когда ревьюер одобрит все доработки.</div><h2>Описание проекта</h2><div class="paragraph">Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. </div><div class="paragraph">Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.</div><div class="paragraph">Постройте модель со значением метрики качества <em>F1</em> не меньше 0.75. </div><h3>Инструкция для проекта</h3><div class="paragraph">Решить задачу можно как с помощью BERT, так и без этой нейронки. Если хотите попробовать BERT —</div><ul><li>Выполните проект локально. В тренажере тетрадь Jupyter ограничена 4 ГБ оперативной памяти — для проекта с BERT этого может не хватить.</li><li>Упомяните BERT в заголовке проекта в первой ячейке:</li></ul><div class="paragraph"><div class="downloadable-image"><a class="downloadable-image__button" download="Image.png"><svg class="icon icon-arrows-24-download downloadable-image__icon" width="24" height="24" viewBox="0 0 24 24" fill="none"><path fill-rule="evenodd" clip-rule="evenodd" d="M12 3C11.45 3 11 3.45 11 4V10.9219C11 11.6763 11.0854 12.4276 11.254 13.1613L11.0483 13.3684L10.8331 13.0242C10.4323 12.3835 9.96022 11.7902 9.42583 11.2558L8.46 10.29C8.07 9.89999 7.44 9.89999 7.05 10.29C6.66 10.68 6.66 11.32 7.05 11.71L10.9404 15.5926C11.526 16.1769 12.474 16.1769 13.0596 15.5926L16.95 11.71C17.34 11.32 17.34 10.68 16.95 10.29C16.56 9.89999 15.93 9.89999 15.54 10.29L14.5742 11.2558C14.0398 11.7902 13.5677 12.3835 13.1669 13.0242L12.9517 13.3684L12.746 13.1613C12.9146 12.4276 13 11.6763 13 10.9219V4C13 3.45 12.55 3 12 3ZM7 19C6.44772 19 6 19.4477 6 20C6 20.5523 6.44772 21 7 21H17C17.5523 21 18 20.5523 18 20C18 19.4477 17.5523 19 17 19H7Z" fill="currentColor" fill-opacity="0.85"></path></svg></a><img src="https://pictures.s3.yandex.net:443/resources/Untitled_55_1610551674.png" alt="image" crossorigin="anonymous" class="image image_expandable"></div></div><div class="paragraph">Выполнить проект без BERT можно локально или в нашем тренажёре.</div><div class="paragraph">В любом случае алгоритм решения выглядит так:</div><ol start="1"><li>Загрузите и подготовьте данные.</li><li>Обучите разные модели.</li><li>Сделайте выводы.</li></ol><h3>Описание данных</h3><div class="paragraph">Данные находятся в файле <code class="code-inline code-inline_theme_light">/datasets/toxic_comments.csv</code>. <a href="https://code.s3.yandex.net/datasets/toxic_comments.csv" target="_blank">Скачать датасет</a>. </div><div class="paragraph">Столбец <em>text</em> в нём содержит текст комментария, а <em>toxic</em> — целевой признак.</div></div>

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import re

import torch
import torch.nn as nn
from torch.optim import Adam

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
import random
from tqdm import tqdm
import numpy as np

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv("/datasets/toxic_comments.csv").drop("Unnamed: 0", axis=1)

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words("english")
stop_words = ["".join(re.sub(r'[^\w\s]', ' ', i).lower().split()) for i in stop_words]

In [ ]:
def tokenizer(string:str) -> str:
    string = re.sub(r'[^a-zA-Z]', ' ', string).lower()
    string = re.sub(r'\b(\w+)(?:\s+\1\b)+', '', string)
    string = re.sub(r'\b(\w*(\w)\2\w*)\b|\b(\w+)\b(?:\s+\3\b)+', '', string).split()
    stems = [lemmatizer.lemmatize(i, pos="n") for i in string]
    stems = [lemmatizer.lemmatize(i, pos="v") for i in stems]
    stems = [lemmatizer.lemmatize(i, pos="a") for i in stems]
    stems = [lemmatizer.lemmatize(i, pos="r") for i in stems]
    stems = [lemmatizer.lemmatize(i, pos="s") for i in stems]
    stems = " ".join([i for i in stems if i not in stop_words])
    return stems

In [ ]:
tqdm.pandas()
data["text"] = data["text"].progress_apply(lambda x: tokenizer(x))
                                  
nans = []
for i, v in enumerate(data["text"]):
    if len(v) == 0:
        nans.append(i)

data = data.drop(nans, axis=0).reset_index(drop=True)

features = data["text"]
target = data["toxic"]

In [ ]:
features.loc[0]

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(12345)

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=12345, stratify=target, test_size=0.33)
features_test, features_val, target_test, target_val = train_test_split(features_test, target_test, random_state=12345, stratify=target_test, test_size=0.33)

In [ ]:
vectorizer = CountVectorizer()
vecs = vectorizer.fit_transform(features_train)
unique_words = vectorizer.get_feature_names_out()

In [ ]:
unique_words 

In [ ]:
features_train = pd.DataFrame(features_train).reset_index(drop=True)
features_test = pd.DataFrame(features_test).reset_index(drop=True)
features_val = pd.DataFrame(features_val).reset_index(drop=True)
target_train = pd.DataFrame(target_train).reset_index(drop=True)
target_test = pd.DataFrame(target_test).reset_index(drop=True)
target_val = pd.DataFrame(target_val).reset_index(drop=True)

In [ ]:
dict_tokenizer = {k:v for v, k in enumerate(unique_words)}
dict_tokenizer["unknown"] = len(dict_tokenizer)
vocab_size = len(dict_tokenizer) + 1

In [ ]:
def tokenization(string:str) -> torch.tensor:
    lbd = lambda x: dict_tokenizer[x] if x in dict_tokenizer else dict_tokenizer["unknown"]
    string = [lbd(i) for i in string.split()]
    return torch.tensor(string, dtype=torch.int32).to(device)

In [ ]:
features_train["text"] = features_train["text"].apply(lambda x: tokenization(x))
features_test["text"] = features_test["text"].apply(lambda x: tokenization(x))
features_val["text"] = features_val["text"].apply(lambda x: tokenization(x))
target_train["toxic"] = target_train["toxic"].apply(lambda x: torch.tensor([x], dtype=torch.float32).to(device))

In [ ]:
class RNN_LSTM(nn.Module):
    def __init__(self, vocab_size, embeddings_len):
        super(RNN_LSTM, self).__init__()

        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embeddings_len)
        self.lstm = nn.LSTM(input_size=embeddings_len, hidden_size=embeddings_len//2, num_layers=5, batch_first=True)
        self.fc_in = nn.Linear(embeddings_len//2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.embeddings(x)
        out, _ = self.lstm(out)
        out = self.fc_in(out[-1])
        out = self.sigmoid(out)
        return out

In [ ]:
model_lstm = RNN_LSTM(vocab_size, 256).to(device)
citeration = nn.BCELoss().to(device)
optimazer = Adam(model_lstm.parameters(), lr=0.0001)

In [ ]:
num_epochs = 3

for _ in range(num_epochs):
    model_lstm.train()
    total_loss = 0
    for features, target in tqdm(zip(features_train["text"], target_train["toxic"])):
        optimazer.zero_grad()
        input = model_lstm(features)
        loss = citeration(input, target)
        loss.backward()
        optimazer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(features_train)
    print(f'Epoch [{_+1}/{num_epochs}], Loss: {average_loss:.4f}')

model_lstm.eval()

In [ ]:
for g in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    predictions = []
    treash_hold = g

    for i in tqdm(features_val["text"]):
        prediction = model_lstm(i)
        prediction = 1 if prediction >= treash_hold else 0
        predictions.append(prediction)

    acc_lstm = accuracy_score(target_val["toxic"], predictions)
    f1_lstm = f1_score(target_val["toxic"], predictions)
    precision_lstm = precision_score(target_val["toxic"], predictions)
    recall_lstm = recall_score(target_val["toxic"], predictions)

    print(f"""Accuracy: {acc_lstm:.4f}
    F1: {f1_lstm:.4f}
    Precision: {precision_lstm:.4f}
    Recall: {recall_lstm:.4f}""")

In [ ]:
class RNN_GRU(nn.Module):
    def __init__(self, vocab_size, embeddings_len):
        super(RNN_GRU, self).__init__()

        self.embeddings = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embeddings_len)
        self.lstm = nn.GRU(input_size=embeddings_len, hidden_size=embeddings_len//2, num_layers=5, batch_first=True)
        self.fc_in = nn.Linear(embeddings_len//2, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        out = self.embeddings(x)
        out, _ = self.lstm(out)
        out = self.fc_in(out[-1])
        out = self.sigmoid(out)
        return out

In [ ]:
model_gru = RNN_GRU(vocab_size, 256).to(device)
citeration = nn.BCELoss().to(device)
optimazer = Adam(model_gru.parameters(), lr=0.0001)

In [ ]:
num_epochs = 3

for _ in range(num_epochs):
    model_gru.train()
    total_loss = 0
    for features, target in tqdm(zip(features_train["text"], target_train["toxic"])):
        optimazer.zero_grad()
        input = model_gru(features)
        loss = citeration(input, target)
        loss.backward()
        optimazer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(features_train)
    print(f'Epoch [{_+1}/{num_epochs}], Loss: {average_loss:.4f}')
model_gru.eval()

In [ ]:
for g in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    predictions = []
    treash_hold = g

    for i in tqdm(features_val["text"]):
        prediction = model_gru(i)
        prediction = 1 if prediction >= treash_hold else 0
        predictions.append(prediction)

    acc_gru = accuracy_score(target_val["toxic"], predictions)
    f1_gru = f1_score(target_val["toxic"], predictions)
    precision_gru = precision_score(target_val["toxic"], predictions)
    recall_gru = recall_score(target_val["toxic"], predictions)

    print(f"""Accuracy: {acc_gru:.4f}
    F1: {f1_gru:.4f}
    Precision: {precision_gru:.4f}
    Recall: {recall_gru:.4f}
    Treash_hold: {treash_hold:.4f}""")

In [ ]:
predictions = []
treash_hold = 0.48

for i in tqdm(features_test["text"]):
    prediction = model_lstm(i)
    prediction = 1 if prediction >= treash_hold else 0
    predictions.append(prediction)

acc_lstm = accuracy_score(target_test["toxic"], predictions)
f1_lstm = f1_score(target_test["toxic"], predictions)
precision_lstm = precision_score(target_test["toxic"], predictions)
recall_lstm = recall_score(target_test["toxic"], predictions)

print(f"""Accuracy: {acc_lstm:.2f}
F1: {f1_lstm:.2f}
Precision: {precision_lstm:.2f}
Recall: {recall_lstm:.2f}""")